In [14]:
import shutil
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [15]:
from PIL import Image

In [3]:
from resnet import resnet18, resnet34, resnet50

for dir in ['youth', 'middle', 'mature', 'old']:
    if not os.path.exists('./train_v2/'+dir):
        os.mkdir('./train_v2/'+dir)

youth = 28 #18~29 

mature = 38 

middle = 49 #30 ~ 49

old = 70 # ~70

DATA_DIR = "./train/"
DES_DIR = "./train_v2/"
age = 18
while(age <= old):
    data_dir = f'{DATA_DIR}{str(age)}'
    for filename in sorted(os.listdir(data_dir)):
        source = data_dir + '/' +filename
        if age <= youth:
            des = DES_DIR +'youth/'+str(age)+'_'+filename
            
        elif age > youth and age <= mature:
            des = DES_DIR +'mature/'+str(age)+'_'+filename
            
        elif age > mature and age <= middle:
            des = DES_DIR +'middle/'+str(age)+'_'+filename
            
        else:
            des = DES_DIR +'old/'+str(age)+'_'+filename
            
        shutil.copy(source, des)
    print(source, des)
        
    age = age + 1


In [4]:
len(os.listdir('./train_v2/youth')), len(os.listdir('./train_v2/middle')) , len(os.listdir('./train_v2/mature')), len(os.listdir('./train_v2/old'))

(17707, 14492, 17074, 7132)

In [16]:
MODEL = resnet50()

In [17]:
MODEL

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2_x): Sequential(
    (0): BottleNeck(
      (residual_function): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stri

In [18]:
import csv
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import pandas as pd


import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torchvision.transforms import functional as F

In [19]:
GPU_NUM = 0
IS_CUDA = torch.cuda.is_available()
#DEVICE  = 'cpu'
DEVICE = torch.device('cuda:' + str(GPU_NUM) if IS_CUDA else 'cpu')

In [20]:
DATASET_DIR = './train'
BATCH_SIZE  = 40

LEARNING_RATE  = 0.00008
L2WEIGHT_DECAY = 0.001
EPOCHS = 50

SaveModelName = "Woman_age_test"

#ModelSavePath = "./Saved_model/" + SaveModelName + "/"
#if not os.path.isdir(ModelSavePath):
#    os.mkdir(ModelSavePath)

In [21]:
class DataManager():
    def __init__(self, dataset_dir, batch_size):
        self.batch_size  = batch_size
        self.dataset_dir = dataset_dir
    def Load_Dataset(self):
        transform =  transforms.Compose([                               
        transforms.Resize([64, 64]),                       
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        dataset = ImageFolder(self.dataset_dir, transform=transform)
        print(f'Total data length : {len(dataset)}')
        test_size = len(dataset)//5
        train_size = len(dataset) - test_size

        #train_data, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])
        train_data, test_data = random_split(dataset, [train_size, test_size])

        print(f'train data size : {len(train_data)} , test data size :{len(test_data)}')
        return train_data, test_data
    
    def Load_DataLoader(self, train, test):
        return DataLoader(train, num_workers=16, batch_size=self.batch_size, shuffle=True, pin_memory=True, drop_last=False), \
                DataLoader(test, num_workers=16, batch_size=self.batch_size, shuffle=True, pin_memory=True, drop_last=False)



In [22]:
from torchvision.models.resnet import resnet18 as _resnet18


N_CLASSES = 53#4#18 ~ 70
EPOCHS = 80
def train_model(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)
    
    DM = DataManager(DATASET_DIR, BATCH_SIZE)
    TRAIN_DATA, TEST_DATA = DM.Load_Dataset()
    TRAIN_LOADER, TEST_LOADER = DM.Load_DataLoader(TRAIN_DATA, TEST_DATA)
    
    #MODEL = _resnet18(pretrained = True)
    MODEL = resnet50()
    MODEL.fc = nn.Linear(MODEL.fc.in_features, 1) #뒷부분 짤라내기

    MODEL = nn.DataParallel(MODEL, output_device=0)
    optimizer = optim.Adam(MODEL.parameters(), lr = LEARNING_RATE, weight_decay = L2WEIGHT_DECAY)
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    MODEL.to(DEVICE)
    total_loss = []
    total_acc = []
  
    for epoch in range(1, EPOCHS):
        MODEL.train()
        #print(f'epoch {epoch} starting')
        with tqdm(TRAIN_LOADER, unit='batch') as train_epoch:
            for i, (inputs, targets) in enumerate(train_epoch):
                #print(inputs.shape)
                inputs = inputs.to(DEVICE)
                targets = targets.to(DEVICE)
                outputs = MODEL(inputs)
                loss = criterion(outputs, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_epoch.set_description(f'Training epoch {epoch}->')
        MODEL.eval()
        with torch.no_grad():
            with tqdm(TEST_LOADER, unit='batch') as test_epoch:
                for j, (val_inputs, val_targets) in enumerate(test_epoch):
                    #print(val_inputs.shape)
                    val_inputs = val_inputs.to(DEVICE)
                    val_targets = val_targets.to(DEVICE)
                    val_outputs = MODEL(val_inputs)
                    val_loss = criterion(val_outputs, val_targets)
                    val_targets_np =  val_targets.to('cpu').detach().numpy()
                    val_outputs_np = val_outputs.to('cpu').detach().numpy()
                    val_outputs_np = np.argmax(val_outputs_np, axis =1).reshape(-1,1)
                    val_acc  = metrics.accuracy_score(val_targets_np, val_outputs_np)
                    total_loss.append(val_loss.cpu().detach().numpy())
                    total_acc.append(val_acc)
                    test_epoch.set_description(f'Evaluating...->')
        total_loss_mean = np.mean(total_loss)
        total_acc_mean = np.mean(total_acc)
        print(f'loss {total_loss_mean:.3f} | Acc {total_acc_mean:.3f}\n') 
        if epoch// 2 == 0:
            torch.save({
            'epoch' : epoch,
            'model_state_dict': MODEL.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss_mean,
            'accuracy': total_acc_mean
            }, f"{checkpoint_dir}/last_checkpoint.pth")

        
        
def evaluate_model(checkpoint_dir):
    DM = DataManager(DATASET_DIR, BATCH_SIZE)
    TRAIN_DATA, TEST_DATA = DM.Load_Dataset()
    TRAIN_LOADER, TEST_LOADER = DM.Load_DataLoader(TRAIN_DATA, TEST_DATA)
    
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    MODEL = _resnet18(pretrained = False)
    MODEL.fc = nn.Linear(MODEL.fc.in_features, N_CLASSES)
    MODEL = nn.DataParallel(MODEL, output_device=0)
    
    LOAD_MODEL = True
    if LOAD_MODEL:
        checkpoint = torch.load(f"{checkpoint_dir}/last_checkpoint.pth")
        MODEL.load_state_dict(checkpoint["model_state_dict"])
    MODEL.to(DEVICE)
    MODEL.eval()
    result_pred, result_annotations = [], []
    LOSS_TRACE,  LOSS_TRACE_FOR_TEST = [], []

    with torch.no_grad():
        with tqdm(TEST_LOADER, unit='batch') as test_epoch:
            for val_inputs, val_targets in test_epoch:
                #print(val_inputs.shape, val_targets.shape)#, len(groups))
                val_inputs = val_inputs.to(DEVICE)
                val_targets = val_targets.to(DEVICE)
                val_outputs = MODEL(val_inputs)

                val_loss = criterion(val_outputs, val_targets)
                val_loss_np = val_loss.cpu().detach().numpy()
                LOSS_TRACE_FOR_TEST.append(val_loss_np)

                y_pred_test_np  = val_outputs.to('cpu').detach().numpy()
                y_pred_test_np  = np.argmax(y_pred_test_np, axis=1).reshape(-1, 1)
                y_test_np       = val_targets.to('cpu').detach().numpy().reshape(-1, 1)

                result_pred.extend(list(y_pred_test_np))
                result_annotations.extend(list(y_test_np))

                test_epoch.set_description(f'Evaluating...->')

    result_pred_np = np.array(result_pred).reshape(-1, 1)
    result_anno_np = np.array(result_annotations).reshape(-1, 1)
    print('--------------------------------------------------------------')
    ACC_TEST = metrics.accuracy_score(result_anno_np, result_pred_np)
    print('Accuracy: ', ACC_TEST)
    conf_mat = metrics.confusion_matrix(result_anno_np, result_pred_np)
    print(metrics.classification_report(result_anno_np, result_pred_np))
    print('Confustion Matrix: ')
    print(conf_mat)

    plt.rcParams["figure.figsize"] = (N_CLASSES, N_CLASSES)
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
    plt.show()
    conf_mat_sum = np.sum(conf_mat, axis=1)
    conf_mat_sum = np.reshape(conf_mat_sum, (N_CLASSES, 1))
    sns.heatmap(conf_mat/conf_mat_sum, annot=True, fmt='.2%', cmap='Blues')
    plt.show()
    print('--------------------------------------------------------------\n\n')      
        
        


In [23]:
transform =  transforms.Compose([                               
        transforms.Resize([256, 256]),                       
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
test = Image.open('./train/18/989.jpg')
transform(test)

tensor([[[0.4275, 0.4275, 0.4235,  ..., 0.2235, 0.2196, 0.2196],
         [0.4275, 0.4275, 0.4235,  ..., 0.2235, 0.2196, 0.2196],
         [0.4275, 0.4275, 0.4235,  ..., 0.2196, 0.2157, 0.2157],
         ...,
         [0.6902, 0.6902, 0.6980,  ..., 0.1451, 0.1451, 0.1451],
         [0.7020, 0.7020, 0.6980,  ..., 0.1451, 0.1451, 0.1451],
         [0.7059, 0.7059, 0.6980,  ..., 0.1451, 0.1451, 0.1451]],

        [[0.4902, 0.4902, 0.4863,  ..., 0.2353, 0.2235, 0.2235],
         [0.4902, 0.4902, 0.4863,  ..., 0.2353, 0.2235, 0.2235],
         [0.4902, 0.4902, 0.4863,  ..., 0.2314, 0.2196, 0.2196],
         ...,
         [0.6039, 0.6039, 0.6118,  ..., 0.1412, 0.1451, 0.1451],
         [0.6157, 0.6157, 0.6118,  ..., 0.1412, 0.1451, 0.1451],
         [0.6196, 0.6196, 0.6118,  ..., 0.1412, 0.1451, 0.1451]],

        [[0.4510, 0.4510, 0.4471,  ..., 0.2392, 0.2314, 0.2314],
         [0.4510, 0.4510, 0.4471,  ..., 0.2392, 0.2314, 0.2314],
         [0.4510, 0.4510, 0.4471,  ..., 0.2353, 0.2275, 0.

In [24]:
# Tests
if __name__ == "__main__":
    checkpoint_dir = f'test_run'
    print("test_train")
    train_model(checkpoint_dir)
    #evaluate_model(checkpoint_dir)

test_train
Total data length : 56405
train data size : 45124 , test data size :11281


RuntimeError: CUDA error: device-side assert triggered

In [ ]:
# Tests
if __name__ == "__main__":
    checkpoint_dir = f'test_run'
    print("test_train")
    #train_model(checkpoint_dir)
    evaluate_model(checkpoint_dir)